In [ ]:
# import settings and functions
%run ./../../imports.ipynb


## What mesh?

Copy your choice to the next cell

for SquareTop:
```
analytical_solution_tag = "-ana_square_top"
generate_config = generateConfig_squareTop
generate_mesh = generateMesh_squareTop
```

for SquareSinCos:
```
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos
```

In [2]:
# Change according to instruction above
analytical_solution_tag = "-ana_mexi_hat"
generate_config = generateConfig_squareMexiHat
generate_mesh = generateMesh_squareMexiHat

# analytical_solution_tag = "-ana_square_top"
# generate_config = generateConfig_squareTop
# generate_mesh = generateMesh_squareTop

# analytical_solution_tag = "-ana_square_sincos"
# generate_config = generateConfig_squareSinCos
# generate_mesh = generateMesh_squareSinCos

## Analysis setup

In [3]:
# which executable?

exe = hdiv_diffusion
sumanalys = "sumanalys.csv"
ana_name = "ana_square_mexi_mixed_order"

ana_compare_exe = [hdiv_diffusion, classic_diffusion]
ana_compare_name = ["ana_square_mexi_mixed", "ana_square_mexi_classic"]
# ana_compare_name = ["ana_square_mexi_mixed"]
ana_compare_sum = ["sumanalys.csv", "FEM_errors.csv"]

# Convergence analysis parameters
order_list = [1, 2, 3, 4] # approximation order p
elem_size_list = [0.1, 0.05, 0.02, 0.01] # element size h
# order_list = [1, 2, 3] # approximation order p
# elem_size_list = [0.5, 0.2, 0.1] # element size h
params.triangle_mesh = True
params.nproc = 1 # number of processors
jumps = ""
if params.nproc == 1:
    jumps = "-get_jumps"
# jumps = "-get_jumps"

run_test = True
run_analysis = True
run_refinement_analysis = True
run_refinement_mesh_analysis = True
run_refinement_hp_analysis = True

run_test = False
run_analysis = False
run_refinement_analysis = False
run_refinement_mesh_analysis = False
run_refinement_hp_analysis = False

naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]
# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

In [4]:
params.conductivity = 1.0 # linear conductivity
params.element_size = elem_size_list[1] # element size in the regular mesh
params.order = 1 # approximation order for displacements

# params.triangle_mesh = False # use triangular mesh

# Pre-processing parameters
params.mesh_file = "square_mexi"
params.length_x = 1
params.length_y = 1
params.length_z = 0
params.show_mesh = True


# solution parameters
params.log_file = "log" # log file name 


## Run test

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [6]:
# Testing mesh generation
if run_test:
    params.show_mesh = True
    generate_config(params)
    generate_mesh(params)

In [7]:
# Testing running analysis
params.order = 3
if run_test:
    !rm out*
    params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    !{mofem_part} -my_file {params.mesh_file + ".h5m"} -nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
    !mpirun -np {params.nproc} {exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} {jumps}

    !convert.py out*


In [8]:
if run_test:
    params.show_file = "out_resu"
    params.show_field = "T"
    params.warp_field_scalar = "T"
    params.show_edges = False
    params.p_cmap = color_temperature
    params.clim = (0, 1)
    params.warp_factor = 0.8
    params.p_resolution = (600, 400)
    params.font_page_part = 1./2.
    params.show_scalar_bar = True
    params.camera_position =  [
            (-1.3, -1.3, 1.1),
            (0.35, 0.35, -0.2),
            (0.25, 0.25, 1),
        ]
    # params.p_cmap = "jet"
    params.p_save = "c3_T_ord_"+str(params.order)+".pdf"
    show_results(params)


    params.clim = None
    params.camera_position = "xy"
    params.warp_field_scalar = ""


In [9]:
def p_settings(p, params):
    if not params.p_resolution:
        params.p_resolution = p.window_size
    else:
        p.window_size = params.p_resolution
    print(f"Current resolution: {params.p_resolution}")
    font_size = int(params.p_resolution[0] * 0.02 / params.font_page_part)  # Adjust the scaling factor as needed

    scalar_bar_title = params.show_field_name if params.show_field_name else params.show_field
    # Define the arguments for the scalar bar
    scalar_bar_args = {
        # 'n_labels': 8,
        # 'position_x': 0.2,  
        'title': scalar_bar_title,
        'title_font_size': font_size,
        'label_font_size': font_size,
        "vertical": True,    
        "position_x": 0.85,     # Shift scalar bar to the right to create margin
        "position_y": 0.2,      # Adjust y-position to lower it slightly
        'fmt': '%.3g'       
    }

    return scalar_bar_args

In [10]:
if run_test:
    params.field_part = 10
    params.show_file = "out_resu"
    params.show_field = "T"
    params.show_field_name = "Grad T"
    params.show_edges = True
    params.clim = [0, 10]
    # params.p_resolution = (1200, 800)
    params.p_resolution = None
    # params.clim = None
    params.p_cmap = color_gradient
    params.p_save = "c3_grad_T.pdf"
    show_results(params)
    params.field_part = -1
    params.show_field_name = None

In [11]:
if run_test:
    params.field_part = -1
    params.show_file = "out_resu"
    params.show_field = "Q"
    params.show_edges = True
    params.p_cmap = color_gradient
    params.p_save = "c3_Q.pdf"
    show_results(params)

In [12]:
if run_test:
    params.show_file = "out_error"
    params.show_field = "ERROR_INDICATOR_GRAD"
    params.show_edges = True
    params.clim = None
    params.warp_field_scalar = ""
    params.p_cmap = "jet"
    # params.warp_factor = 0.4  # warp factor
    params.p_save = "c3_err_est_grad.pdf"
    show_results(params)

In [13]:
if run_test:
    params.show_file = "out_error"
    params.show_field = "ERROR_FLUX"
    params.show_edges = True
    # params.p_cmap = "jet"
    params.p_save = "c3_err_flux.pdf"
    show_results(params)

In [14]:
if jumps and run_test:
    params.show_file = "out_error"
    params.show_field = "JUMP_L2"
    params.show_edges = True
    params.p_cmap = "jet"
    params.p_save = "c3_err_ind_jump.pdf"
    show_results(params)

In [15]:
if run_test:
    params.show_file = "out_error"
    params.show_field = "ERROR_ESTIMATOR"
    params.show_edges = True
    params.p_cmap = "jet"
    params.p_save = "c3_err_est.pdf"
    show_results(params)

In [16]:

if run_test:
    params.show_file = "out_error"
    params.show_field = "ERROR_H1_SEMINORM"
    params.show_edges = True
    params.p_save = "c3_err_H1.pdf"
    show_results(params)

In [17]:
# if run_test:
#     # params.show_file = "out_ori_result"
#     params.show_file = "out_result"
#     params.show_field = "P_reference"
#     params.warp_field_scalar = "P_reference"
#     params.warp_factor = 0.4  # warp factor
#     params.show_edges = True
#     params.p_save = "run_test_p.pdf"
#     show_results(params)

In [18]:
if run_test:
    params.show_file = "out_error"
    params.show_field = "ERROR_INDICATOR_DIV"
    params.show_edges = True
    params.warp_field_scalar = ""
    # params.warp_factor = 0.4  # warp factor
    params.p_save = "c3_test_err_ind_div.pdf"
    show_results(params)

In [19]:
if run_test:
    params.show_file = "out_error"
    params.show_field = "ERROR_INDICATOR_GRAD"
    params.show_edges = True
    params.warp_field_scalar = ""
    # params.warp_factor = 0.4  # warp factor
    params.p_save = "c3_test_err_ind_grad.pdf"
    show_results(params)

## Comparison between standard and mixed

In [20]:
if run_analysis:    
    for i in range(len(ana_compare_name)):
        !rm {ana_compare_sum[i]}
        !rm ./out_*
        for elem_size in elem_size_list:
            params.element_size = elem_size
            params.show_mesh = False
            generate_mesh(params)
            params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
            !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
            for order in order_list:
                params.order = order
                !mpirun -np {params.nproc} {ana_compare_exe[i]} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} {jumps}
        !mv {ana_compare_sum[i]} {ana_compare_name[i]}.csv
    


In [21]:
# refinement analysis

ref_iter_num = 7
ref_control = 2.0

In [22]:
exe = hdiv_diffusion
sumanalys = "sumanalys.csv"
ana_ref_ord_name = "ana_square_mexi_mixed_order"

refinement_style = 1
# ref_iter_num = 7
# ref_control = 5.0
params.nproc = 1

if run_refinement_analysis:    
    !rm {sumanalys}
    !rm ./out_*
    elem_size =  elem_size_list[1]
    params.element_size = elem_size
    params.show_mesh = True
    generate_mesh(params)
    params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
    order = order_list[0]
    params.order = order
    !mpirun -np {params.nproc} {exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -refinement_style {refinement_style} -ref_iter_num {ref_iter_num} -ref_control {ref_control} {jumps}
    !mv {sumanalys} {ana_ref_ord_name}.csv

In [ ]:
!convert.py out*

if run_test:
    params.show_file = "out_error"
    params.show_field = "ORDER"
    params.show_edges = True
    params.p_cmap = "rainbow"
    params.p_save = "c3_order.pdf"
    # params.p_save = "run_test_err_ind_grad.pdf"
    show_results(params)

    params.show_file = "out_error"
    params.show_field = "ERROR_INDICATOR_GRAD"
    params.show_edges = True
    params.p_cmap = "jet"
    params.p_save = ""
    # params.p_save = "run_test_err_ind_grad.pdf"
    show_results(params)

In [24]:
exe = hdiv_diffusion
sumanalys = "sumanalys.csv"
ana_ref_mesh_name = "ana_square_mexi_mixed_mesh"

if run_refinement_mesh_analysis:   
    refinement_style = 2
    # ref_iter_num = 7
    # ref_control = 2.0
    params.nproc = 1

    !rm {sumanalys}
    !rm ./out_*
    elem_size =  elem_size_list[1]
    params.element_size = elem_size
    params.show_mesh = True
    generate_mesh(params)
    params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
    order = order_list[0]
    params.order = order
    !mpirun -np {params.nproc} {exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -refinement_style {refinement_style} -ref_iter_num {ref_iter_num} -ref_control {ref_control} {jumps}
    !mv {sumanalys} {ana_ref_mesh_name}.csv

In [ ]:
!convert.py out*

if run_refinement_mesh_analysis:
    params.show_file = "out_error"
    params.show_field = "ERROR_INDICATOR_DIV"
    params.show_edges = True
    params.p_cmap = "jet"
    # params.p_save = "run_test_err_ind_grad.pdf"
    show_results(params)
    params.show_field = "ERROR_ESTIMATOR"
    show_results(params)

In [26]:
# exe = hdiv_diffusion
# sumanalys = "sumanalys.csv"
ana_ref_hp_name = "ana_square_mexi_mixed_hp"

if run_refinement_hp_analysis:   
    refinement_style = 3
    # ref_iter_num = 7
    # ref_control = 2.0
    params.nproc = 1

    !rm {sumanalys}
    !rm ./out_*
    # elem_size =  elem_size_list[1]
    # order = order_list[1]
    elem_size =  elem_size_list[0]
    order = order_list[0]

    params.element_size = elem_size
    params.show_mesh = True
    generate_mesh(params)
    params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
    
    params.order = order
    !mpirun -np {params.nproc} {exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -refinement_style {refinement_style} -ref_iter_num {ref_iter_num} -ref_control {ref_control} {jumps}
    !mv {sumanalys} {ana_ref_hp_name}.csv

In [ ]:
!convert.py out*

if run_refinement_hp_analysis:
    params.show_file = "out_error"
    params.show_field = "ERROR_INDICATOR_DIV"
    params.show_edges = True
    params.p_cmap = "jet"
    # params.p_save = "run_test_err_ind_grad.pdf"
    show_results(params)


    params.show_field = "ORDER"
    show_results(params)
    params.show_field = "ERROR_H1_SEMINORM"
    show_results(params)


### Load analysis

In [ ]:
print(naming)

In [29]:
error_name_list = []
error_label_list = []

error_name_list.append("L2norm")
error_label_list.append(r'Global error $L^2$-norm')
error_name_list.append("H1seminorm")
error_label_list.append(r'Global error $H^1$-seminorm')
error_name_list.append("fluxErr")
error_label_list.append(r'Global Flux error')

error_name_list.append("errorEstimator")
error_label_list.append(r'Global error estimator')
error_name_list.append("errorIndicatorGrad")
error_label_list.append(r'Global error indicator grad')
error_name_list.append("errorIndicatorDiv")
error_label_list.append(r'Global error indicator div')

if jumps:
    error_name_list.append("jumpL2")
    error_label_list.append(r'Global jump L2')
    # error_name_list.append("jumpHdiv")
    # error_label_list.append(r'Global jump Hdiv')



In [30]:
filename_prefix = "c3_mixed_hat_"

mixed_ana = Analysis(ana_compare_name[0], naming, order_list, error_name_list, error_label_list, filename_prefix, elem_size_list,  marker='x', linestyle='--', plot_gradients=True, label="Mixed")
classic_ana = Analysis(ana_compare_name[1], naming, order_list, error_name_list, error_label_list, filename_prefix, elem_size_list,  marker='o', linestyle='-', plot_gradients=False, label="Standard")


order_ref_ana = Analysis(ana_ref_ord_name, naming, order_list, error_name_list, error_label_list, filename_prefix, elem_size_list,  marker='*', linestyle=':', plot_gradients=False, label=r"Adaptive $p$-refinement", color = 'black')

mesh_ref_ana = Analysis(ana_ref_mesh_name, naming, order_list, error_name_list, error_label_list, filename_prefix, elem_size_list,  marker='v', linestyle=':', plot_gradients=False, label=r"Adaptive $h$-refinement", color = 'black')

hp_ref_ana = Analysis(ana_ref_hp_name, naming, order_list, error_name_list, error_label_list, filename_prefix, elem_size_list,  marker='s', linestyle=':', plot_gradients=False, label=r'Adaptive $hp$-refinement', color = 'red')

# ana_ref_ord_name

### Plot results

In [ ]:
# mixed_ana.plot_both_analyses_by_elem_size([classic_ana])
mixed_ana.plot_both_analyses_by_elem_size([])
# mixed_ana.plot_both_analyses_by_gaussnum([classic_ana], [order_ref_ana, mesh_ref_ana])
mixed_ana.plot_both_analyses_by_gaussnum([])

In [ ]:
mixed_ana.plot_gradients = False
mixed_ana.legend_fond_size = 12

mixed_ana.plot_both_analyses_by_gaussnum([classic_ana])
mixed_ana.plot_both_analyses_by_elem_size([classic_ana])

mixed_ana.filename_prefix = "c3_mixed_hat_ref_order_"
mixed_ana.plot_both_analyses_by_gaussnum([], [order_ref_ana])
mixed_ana.filename_prefix = "c3_mixed_hat_ref_mesh_"
mixed_ana.plot_both_analyses_by_gaussnum([], [mesh_ref_ana])
mixed_ana.filename_prefix = "c3_mixed_hat_ref_order_mesh_"
mixed_ana.plot_both_analyses_by_gaussnum([], [order_ref_ana, mesh_ref_ana])
mixed_ana.filename_prefix = "c3_mixed_hat_ref_order_mesh_hp_"
# mixed_ana.plot_both_analyses_by_gaussnum([], [order_ref_ana, mesh_ref_ana, hp_ref_ana])